In [1]:
import numpy as np
import pandas as pd
import os

from helper_classes import *
from utils import *
from similarity_predictor import check_similarity

In [2]:
def get_public_data(input_data_path):
    data1 = pd.read_csv(os.path.join(input_data_path, "UCI_Credit_Card.csv"))
    data2 = pd.read_csv(os.path.join(input_data_path, "Age2_address1_credit_card3.csv"))
    data3 = pd.read_csv(os.path.join(input_data_path, "list_of_real_usa_addresses.csv"))
    data4 = pd.read_csv(os.path.join(input_data_path, "CardBase.csv"))
    data5 = pd.read_csv(os.path.join(input_data_path, "Credit_Card2.csv"))
    data6 = pd.read_csv(os.path.join(input_data_path, "catalog.csv"))
    data7 = pd.read_csv(os.path.join(input_data_path, "iban.csv"))
    data12 = pd.read_csv(
        os.path.join(input_data_path, "2018-seattle-business-districts.csv")
    )
    data13 = pd.read_csv(os.path.join(input_data_path, "Customer_Segmentation.csv"))
    data14 = pd.read_csv(os.path.join(input_data_path, "application_record.csv"))
    data15 = pd.read_csv(os.path.join(input_data_path, "Airbnb_Open_Data.csv"))
    data16 = pd.read_csv(
        os.path.join(input_data_path, "Book1.xlsx-credit-card-number.csv")
    )
    data17 = pd.read_csv(os.path.join(input_data_path, "Aliases.csv"))
    data21 = pd.read_csv(os.path.join(input_data_path, "Emails.csv"))
    data25 = pd.read_csv(os.path.join(input_data_path, "Persons.csv"))
    data27 = pd.read_csv(os.path.join(input_data_path, "Bachelor_Degree_Majors.csv"))
    data28 = pd.read_csv(os.path.join(input_data_path, "CrabAgePrediction.csv"))
    data29 = pd.read_csv(os.path.join(input_data_path, "Salary_Data.csv"))
    return {
        "data1": data1,
        "data2": data2,
        "data3": data3,
        "data4": data4,
        "data5": data5,
        "data6": data6,
        "data7": data7,
        "data12": data12,
        "data13": data13,
        "data14": data14,
        "data15": data15,
        "data16": data16,
        "data17": data17,
        "data21": data21,
        "data25": data25,
        "data27": data27,
        "data28": data28,
        "data29": data29,
    }


In [3]:
dataset_name_mapping = {
        "data1": "UCI_Credit_Card",
        "data2": "Age2_address1_credit_card3",
        "data3": "list_of_real_usa_addresses",
        "data4": "CardBase",
        "data5": "Credit_Card2",
        "data6": "catalog",
        "data7": "iban",
        "data12": "2018-seattle-business-districts",
        "data13": "Customer_Segmentation",
        "data14": "application_record",
        "data15": "Airbnb_Open_Data",
        "data16": "Book1.xlsx-credit-card-number",
        "data17": "Aliases",
        "data21": "Emails",
        "data25": "Persons",
        "data27": "Bachelor_Degree_Majors",
        "data28": "CrabAgePrediction",
        "data29": "Salary_Data",
    }


platforms = ["A","B","C","D","E"]


In [4]:
input_data_dir = "C:\\Glossary_Terms_Git\\datahub-classify\\test\\datasets"
public_data_list = get_public_data(input_data_dir)

In [5]:

def populate_tableinfo_object(dataset_key):
    table_meta_info = {
            "Name": dataset_name_mapping[dataset_key],
            "Description": f"This table contains description of {dataset_name_mapping[dataset_key]}",
            "Platform": platforms[np.random.randint(0,5)],
            "Table_Id": dataset_key
                        }

    col_infos=[]
    for col in public_data_list[dataset_key].columns:
        fields = {
                    'Name': col,
                    'Description': f' {col}',
                    'Datatype': public_data_list[dataset_key][col].dropna().dtype ,
                    'Dataset_Name': dataset_key,
                    'Column_Id': dataset_key + "_" + col
                }
        metadata_col = Metadata(fields)
        parent_cols = []
        col_info = ColumnInfo(metadata_col, parent_cols)
        col_infos.append(col_info)

    metadata_table = Metadata_table(table_meta_info)
    parent_tables= []
    table_info = TableInfo(metadata_table, parent_tables, col_infos)
    return table_info

def populate_similar_tableinfo_object(dataset_key):
    df = public_data_list[dataset_key].copy()
    random_df_key = "data" + str(np.random.randint(1,len(dataset_name_mapping)))
    while not (dataset_name_mapping.get(random_df_key, None)) or (random_df_key == dataset_key):
        random_df_key = "data" + str(np.random.randint(1,len(dataset_name_mapping)))
    random_df = public_data_list[random_df_key].copy()
    
    df.columns = [ dataset_key + "_" + col for col in df.columns]
    random_df.columns = [random_df_key + "_" + col for col in random_df.columns]
    second_df = pd.concat([df, random_df], axis=1)
    cols_to_keep = list(df.columns) + list(random_df.columns[:2])
    second_df = second_df[cols_to_keep]
#     print(second_df.head())

    table_meta_info = {
        "Name": dataset_name_mapping[dataset_key] + "_v2",
        "Description": f" {dataset_name_mapping[dataset_key]}",
        "Platform": platforms[np.random.randint(0,5)],
        "Table_Id": dataset_key + "_" + random_df_key
                    }

    col_infos=[]
    for col in second_df.columns:
        fields = {
                    'Name': col.split("_",1)[1],
                    'Description': f'{col.split("_",1)[1]}',
                    'Datatype': second_df[col].dropna().dtype,
                    'Dataset_Name': dataset_key + "_" + random_df_key,
                    'Column_Id':  col
                }
        metadata_col = Metadata(fields)
        parent_cols = []
        col_info = ColumnInfo(metadata_col, parent_cols)
        col_infos.append(col_info)
    metadata_table = Metadata_table(table_meta_info)
    parent_tables= []
    table_info = TableInfo(metadata_table, parent_tables, col_infos)
    return table_info
    

In [6]:
table_info_1 = populate_tableinfo_object(dataset_key = "data3")
table_info_2 = populate_similar_tableinfo_object(dataset_key = "data3")
overall_table_similarity_score, column_similarity_scores = check_similarity(table_info_1, table_info_2)
# overall_table_similarity_score,column_similarity_scores

finding table similarity
name score:  1.0
desc score:  1.0
platform score:  1
lineae score:  0
schema score:  0.8333333333333334
********************finding column similarities**********************
total pairs -->  35


In [7]:
overall_table_similarity_score,column_similarity_scores

(1.0,
 {('data3_Unnamed: 0', 'data3_Unnamed: 0'): 1.0,
  ('data3_Unnamed: 0', 'data3_address'): 0.27,
  ('data3_Unnamed: 0', 'data3_city'): 0.22,
  ('data3_Unnamed: 0', 'data3_state'): 0.31,
  ('data3_Unnamed: 0', 'data3_zip'): 0.04,
  ('data3_Unnamed: 0', 'data17_Unnamed: 0'): 1.0,
  ('data3_Unnamed: 0', 'data17_Id'): 0.3,
  ('data3_address', 'data3_Unnamed: 0'): 0.27,
  ('data3_address', 'data3_address'): 1.0,
  ('data3_address', 'data3_city'): 0.51,
  ('data3_address', 'data3_state'): 0.65,
  ('data3_address', 'data3_zip'): 0.2,
  ('data3_address', 'data17_Unnamed: 0'): 0.27,
  ('data3_address', 'data17_Id'): 0.3,
  ('data3_city', 'data3_Unnamed: 0'): 0.22,
  ('data3_city', 'data3_address'): 0.51,
  ('data3_city', 'data3_city'): 1.0,
  ('data3_city', 'data3_state'): 0.96,
  ('data3_city', 'data3_zip'): 0.22,
  ('data3_city', 'data17_Unnamed: 0'): 0.22,
  ('data3_city', 'data17_Id'): 0.16,
  ('data3_state', 'data3_Unnamed: 0'): 0.31,
  ('data3_state', 'data3_address'): 0.65,
  ('data

In [8]:
table_info_1 = populate_tableinfo_object(dataset_key = "data3")
table_info_2 = populate_tableinfo_object(dataset_key = "data5")
overall_table_similarity_score, column_similarity_scores = check_similarity(table_info_1, table_info_2)
overall_table_similarity_score,column_similarity_scores

finding table similarity
name score:  0.336
desc score:  0.83
platform score:  1
lineae score:  0
schema score:  0.21052631578947367
********************finding column similarities**********************
total pairs -->  70


(0.37,
 {('data3_Unnamed: 0', 'data5_Unnamed: 0'): 1.0,
  ('data3_Unnamed: 0', 'data5_User'): 0.43,
  ('data3_Unnamed: 0', 'data5_CARD INDEX'): 0.29,
  ('data3_Unnamed: 0', 'data5_Card Brand'): 0.18,
  ('data3_Unnamed: 0', 'data5_Card Type'): 0.23,
  ('data3_Unnamed: 0', 'data5_Card Number'): 0.37,
  ('data3_Unnamed: 0', 'data5_Expires'): 0.09,
  ('data3_Unnamed: 0', 'data5_CVV'): 0.19,
  ('data3_Unnamed: 0', 'data5_Has Chip'): 0.15,
  ('data3_Unnamed: 0', 'data5_Cards Issued'): 0.31,
  ('data3_Unnamed: 0', 'data5_Credit Limit'): 0.18,
  ('data3_Unnamed: 0', 'data5_Acct Open Date'): 0.18,
  ('data3_Unnamed: 0', 'data5_Year PIN last Changed'): 0.37,
  ('data3_Unnamed: 0', 'data5_Card on Dark Web'): 0.18,
  ('data3_address', 'data5_Unnamed: 0'): 0.27,
  ('data3_address', 'data5_User'): 0.45,
  ('data3_address', 'data5_CARD INDEX'): 0.19,
  ('data3_address', 'data5_Card Brand'): 0.45,
  ('data3_address', 'data5_Card Type'): 0.37,
  ('data3_address', 'data5_Card Number'): 0.33,
  ('data3_a

In [9]:
table_info_1 = populate_tableinfo_object(dataset_key = "data2")
table_info_2 = populate_tableinfo_object(dataset_key = "data27")
overall_table_similarity_score, column_similarity_scores = check_similarity(table_info_1, table_info_2)
overall_table_similarity_score,column_similarity_scores

finding table similarity
name score:  0.296
desc score:  0.72
platform score:  0
lineae score:  0
schema score:  0.3448275862068966
********************finding column similarities**********************
total pairs -->  190


(0.32,
 {('data2_Unnamed: 0', 'data27_Unnamed: 0'): 1.0,
  ('data2_Unnamed: 0', 'data27_State'): 0.31,
  ('data2_Unnamed: 0', 'data27_Sex'): 0.21,
  ('data2_Unnamed: 0', 'data27_Age Group'): 0.14,
  ('data2_Unnamed: 0', "data27_Bachelor's Degree Holders"): 0.12,
  ('data2_Unnamed: 0', 'data27_Science and Engineering'): 0.24,
  ('data2_Unnamed: 0', 'data27_Science and Engineering Related Fields'): 0.24,
  ('data2_Unnamed: 0', 'data27_Business'): 0.29,
  ('data2_Unnamed: 0', 'data27_Education'): 0.15,
  ('data2_Unnamed: 0', 'data27_Arts, Humanities and Others'): 0.18,
  ('data2_Person', 'data27_Unnamed: 0'): 0.36,
  ('data2_Person', 'data27_State'): 0.55,
  ('data2_Person', 'data27_Sex'): 0.71,
  ('data2_Person', 'data27_Age Group'): 0.49,
  ('data2_Person', "data27_Bachelor's Degree Holders"): 0.49,
  ('data2_Person', 'data27_Science and Engineering'): 0.49,
  ('data2_Person', 'data27_Science and Engineering Related Fields'): 0.49,
  ('data2_Person', 'data27_Business'): 0.55,
  ('data2_